In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint

# 1. 출력 스키마 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행한 장소 (도시 또는 지역명)"),
    ResponseSchema(name="duration", description="여행 기간 (예: '2박 3일', '3일간')"),
    ResponseSchema(name="budget", description="총 사용한 예산 (예: '30만원')"),
    ResponseSchema(name="rating", description="1~5점 사이의 추천도 점수"),
    ResponseSchema(name="activities", description="주요 활동 리스트 (예: ['바다 구경', '맛집 탐방'])")
]

# 2. 파서 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 3. 프롬프트 정의
template = """
다음 여행 후기에서 핵심 정보를 추출하세요.

후기: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template).partial(
    format_instructions=format_instructions
)

# 4. 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 5. 테스트용 여행 후기 입력
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

# 6. 체인 실행
chain = prompt | model | parser
output = chain.invoke({"review": review})

# 7. 결과 출력
print("===== 여행 후기 분석 결과 =====")
pprint(output)

===== 여행 후기 분석 결과 =====
{'activities': '해운대 바다구경, 자갈치시장에서 회 먹기, 감천문화마을 구경',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4'}
